# **LGBM Model Baseline**
----
### 1. 데이터 전처리

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

import numpy as np
import pandas as pd

# from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
# from konlpy.tag import Twitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# # font_path = '/WINDOWS/FONTS/NanumGothic.ttf'
# # font_name = fm.FontProperties(fname=font_path, size=10).get_name()
# plt.rc('font', family='Malgun Gothic')
# plt.rcParams["figure.figsize"] = (20, 10)
# register_matplotlib_converters()

# mpl.font_manager._rebuild()
# mpl.pyplot.rc('font', family='Malgun Gothic')
# pd.options.mode.chained_assignment = None

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
val = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/val.json', typ = 'frame',encoding='UTF-8')
val.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000


In [ ]:
song_meta = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/song_meta.json', typ = 'frame',encoding='UTF-8')
song_meta.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2


## train

In [ ]:
# id와 세부장르만 추출한 새로운 데이터프레임 만들기
song_gnr_map = song_meta.loc[:, ['id', 'song_gn_dtl_gnr_basket']] 

# unnest 작업
song_gnr_map_unnest = np.dstack(
    (   # repeat(반복할 리스트, 반복 횟수) = id값을 세부장르 길이 만큼 반복??!!!!
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_dtl_gnr_basket))), 
        np.concatenate(song_gnr_map.song_gn_dtl_gnr_basket.values) 
        # concatenate : numpy 배열 합치기
    )
)
song_gnr_map_unnest

array([[['0', 'GN0901'],
        ['1', 'GN1601'],
        ['1', 'GN1606'],
        ...,
        ['707987', 'GN1801'],
        ['707988', 'GN0601'],
        ['707988', 'GN0604']]], dtype='<U32')

In [ ]:
# unnested 데이터프레임 생성 : song_gnr_map

song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)
song_gnr_map['id'] = song_gnr_map['id'].astype(str) # 문자열 변환
song_gnr_map.rename(columns = {'id' : 'song_id', 'song_gn_dtl_gnr_basket' : 'gnr_code'}, inplace = True)

In [ ]:
song_gnr_map.head(20) # song id에 두개 이상의 장르가 매핑된 것도 있음. 주의!
# id를 세부장르 길이만큼 반복했으니까!! 나 이제 이해갔어!!

,song_id,gnr_code
0,0,GN0901
1,1,GN1601
2,1,GN1606
3,2,GN0901
4,3,GN1102
5,3,GN1101
6,4,GN1802
7,4,GN1801
8,5,GN1701
9,6,GN1601


In [ ]:
train = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/train.json', typ = 'frame', encoding='utf-8')
test = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/test.json', typ = 'frame', encoding='utf-8')
genre_gn_all = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/genre_gn_all.json', typ = 'series',encoding='utf-8')

In [ ]:
# train에서 id와 song만 추출

plylst_song_map = train[['id', 'songs']] #id와 song만 추출

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest


In [ ]:
plylst_song_map.head(20) # id별로 songs를 매핑. 한 아이디에 여러개의 songs 매핑됨

,id,songs
0,61281,525514
1,61281,129701
2,61281,383374
3,61281,562083
4,61281,297861
5,61281,139541
6,61281,351214
7,61281,650298
8,61281,531057
9,61281,205238


In [ ]:
song_gnr_map.columns = ['songs', 'gmr_code']
song_gnr_map.head() # 매핑시키려고 칼럼 이름 똑같이 만들어줌!

,songs,gmr_code
0,0,GN0901
1,1,GN1601
2,1,GN1606
3,2,GN0901
4,3,GN1102


### song 과 장르데이터를 매핑시킨다

In [ ]:
# 같은 songs를 중심으로 매핑시킨다!
train_g = pd.merge(plylst_song_map, song_gnr_map, how='left', on='songs').fillna('장르없음')

In [ ]:
train_g.head() # 세부장르까지 나와있음. 그래서 songs가 여러개임. 
# id는 플레이리스트 아이디***

,id,songs,gmr_code
0,61281,525514,GN1402
1,61281,525514,GN1401
2,61281,129701,GN0901
3,61281,129701,GN0902
4,61281,129701,GN1001


In [ ]:
## f1 score구현
# 각 플레이리스트 안에 해당 장르가 몇개씩 있는지? 

f1 = pd.pivot_table(data = train_g, index='id', columns='gmr_code', aggfunc='count',
                    fill_value=0,values='songs',margins=True)
f1

gmr_code,GN0101,GN0102,GN0103,GN0104,GN0105,GN0201,GN0202,GN0203,GN0204,GN0205,GN0301,GN0302,GN0303,GN0304,GN0401,GN0402,GN0403,GN0501,GN0502,GN0503,GN0504,GN0505,GN0506,GN0507,GN0508,GN0509,GN0601,GN0602,GN0603,GN0604,GN0605,GN0606,GN0701,GN0702,GN0703,GN0704,GN0705,GN0706,GN0707,GN0708,...,GN2209,GN2210,GN2211,GN2212,GN2213,GN2214,GN2215,GN2216,GN2301,GN2302,GN2401,GN2402,GN2403,GN2404,GN2405,GN2406,GN2501,GN2502,GN2503,GN2504,GN2505,GN2506,GN2601,GN2602,GN2603,GN2701,GN2702,GN2703,GN2704,GN2801,GN2802,GN2803,GN2804,GN2805,GN2806,GN2901,GN2902,GN2903,장르없음,All
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,5,1,2,2,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14
1000,1,0,0,0,0,3,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27
100000,7,0,0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,15,0,14,1,0,0,0,4,11,16,0,0,0,4,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,105
100001,5,0,0,0,3,0,0,0,0,0,2,0,2,0,4,2,2,6,2,3,0,0,1,0,0,6,3,0,0,0,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123
100002,12,0,0,2,8,1,0,0,0,1,0,0,0,0,3,1,3,2,2,0,0,0,0,0,0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37
99993,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10
99997,1,0,0,0,0,0,0,0,0,0,40,0,26,17,2,2,2,17,0,0,0,17,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,132


In [ ]:
f1 = f1.iloc[:-1, :-1].div(f1.iloc[:-1,-1], axis=0).reset_index() 
# 가로 세로 'ALL'행 제외. (-1)
# div() : 데이터 프레임 안의 요소들을 나눠줌. 전체에서 그 장르가 차지하는 비율로 나눔. 

In [ ]:
f1

gmr_code,id,GN0101,GN0102,GN0103,GN0104,GN0105,GN0201,GN0202,GN0203,GN0204,GN0205,GN0301,GN0302,GN0303,GN0304,GN0401,GN0402,GN0403,GN0501,GN0502,GN0503,GN0504,GN0505,GN0506,GN0507,GN0508,GN0509,GN0601,GN0602,GN0603,GN0604,GN0605,GN0606,GN0701,GN0702,GN0703,GN0704,GN0705,GN0706,GN0707,...,GN2208,GN2209,GN2210,GN2211,GN2212,GN2213,GN2214,GN2215,GN2216,GN2301,GN2302,GN2401,GN2402,GN2403,GN2404,GN2405,GN2406,GN2501,GN2502,GN2503,GN2504,GN2505,GN2506,GN2601,GN2602,GN2603,GN2701,GN2702,GN2703,GN2704,GN2801,GN2802,GN2803,GN2804,GN2805,GN2806,GN2901,GN2902,GN2903,장르없음
0,1,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.357143,0.071429,0.142857,0.142857,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000,0.037037,0.0,0.0,0.000000,0.000000,0.111111,0.0,0.0,0.037037,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.000000,0.037037,0.0,0.037037,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100000,0.066667,0.0,0.0,0.009524,0.047619,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.133333,0.009524,0.000000,0.000000,0.0,0.038095,0.104762,0.152381,0.0,0.0,0.0,0.038095,0.095238,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.009524,0.0,0.009524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100001,0.040650,0.0,0.0,0.000000,0.024390,0.000000,0.0,0.0,0.000000,0.000000,0.016260,0.000000,0.016260,0.000000,0.032520,0.016260,0.016260,0.048780,0.016260,0.024390,0.000000,0.000000,0.008130,0.0,0.000000,0.048780,0.024390,0.0,0.0,0.0,0.000000,0.024390,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100002,0.279070,0.0,0.0,0.046512,0.186047,0.023256,0.0,0.0,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.069767,0.023256,0.069767,0.046512,0.046512,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.046512,0.023256,0.0,0.0,0.0,0.000000,0.023256,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115066,99991,0.025974,0.0,0.0,0.000000,0.019481,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038961,0.019481,0.025974,0.162338,0.097403,0.051948,0.006494,0.000000,0.006494,0.0,0.006494,0.155844,0.051948,0.0,0.0,0.0,0.000000,0.045455,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.006494,0.0,0.006494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115067,99992,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [ ]:
# f1.loc[f1['id']=='1']

In [ ]:
f1['id'] = f1['id'].map(lambda x : int(x)) # id를 정수로 바꾸기

In [ ]:
# 똑같은건가? 
# f1['id'] = f1['id'].astype(int)
# f1.info()

In [ ]:
# f1 score와 train plylst를 합침
train2 = pd.merge(train, f1, how='left', on='id')

In [ ]:
# pandas 가로 출력 길이 늘리기
pd.set_option('display.max.colwidth', 20)

In [ ]:
train2.head(3) 
# train의 플레이리스트 + 그 플레이리스트 안의 song들이 어떤 장르에 많이 속하는지 비율

,tags,id,plylst_title,songs,like_cnt,updt_date,GN0101,GN0102,GN0103,GN0104,GN0105,GN0201,GN0202,GN0203,GN0204,GN0205,GN0301,GN0302,GN0303,GN0304,GN0401,GN0402,GN0403,GN0501,GN0502,GN0503,GN0504,GN0505,GN0506,GN0507,GN0508,GN0509,GN0601,GN0602,GN0603,GN0604,GN0605,GN0606,GN0701,GN0702,...,GN2208,GN2209,GN2210,GN2211,GN2212,GN2213,GN2214,GN2215,GN2216,GN2301,GN2302,GN2401,GN2402,GN2403,GN2404,GN2405,GN2406,GN2501,GN2502,GN2503,GN2504,GN2505,GN2506,GN2601,GN2602,GN2603,GN2701,GN2702,GN2703,GN2704,GN2801,GN2802,GN2803,GN2804,GN2805,GN2806,GN2901,GN2902,GN2903,장르없음
0,[락],61281,여행같은 음악,"[525514, 129701,...",71,2013-12-19 18:36...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945,...",1,2014-12-02 16:19...,0.104762,0.0,0.009524,0.028571,0.057143,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.000,0.066667,0.019048,0.047619,0.0000,0.0,0.0,0.0,0.019048,0.047619,0.07619,0.0,0.0,0.0,0.047619,0.028571,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009524,0.009524,0.009524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있...","[83116, 276692, ...",17,2017-08-28 07:09...,0.025000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0375,0.0,0.0375,0.0,0.1875,0.0125,0.125,0.150000,0.012500,0.050000,0.0125,0.0,0.0,0.0,0.000000,0.150000,0.05000,0.0,0.0,0.0,0.000000,0.050000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012500,0.000000,0.012500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 날씨 열 추가

def weather(x) : 
    if (3 <= x) & (x <6) : 
        return('spring')
    elif (6 <= x) & (x <9) : 
        return('summer')
    elif (9 <= x) & (x <11) : 
        return('autumn')    
    else : return('winter')

In [ ]:
# datetime 으로 나눔. 
train2['year'] = pd.to_datetime(train2['updt_date']).map(lambda x : x.year)
train2['month'] = pd.to_datetime(train2['updt_date']).map(lambda x : x.month)
train2['weather'] = train2['month'].apply(weather)

In [ ]:
# 원핫 인코딩 
train2 = pd.get_dummies(data=train2,columns=['weather']) 

In [ ]:
# 빈 리스트를 nan으로 바꿈

def list_to_nan(x) : 
  if x == [] :
    return (np.nan)
  else : return x

In [ ]:
train['songs'] = train['songs'].apply(list_to_nan)
val['songs'] = val['songs'].apply(list_to_nan)

In [ ]:
train = train.dropna()
val = val.dropna()

## Valid

In [ ]:
plylst_song_map = val[['id', 'songs']]

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

song_gnr_map.columns = ['songs','gmr_code']

val_g = pd.merge(plylst_song_map,song_gnr_map,how='left',on='songs').fillna('장르없음')

f1 = pd.pivot_table(data = val_g,index='id',columns='gmr_code',aggfunc='count',fill_value=0,values='songs',margins=True)

f1 = f1.iloc[:-1,:-1].div(f1.iloc[:-1,-1], axis=0).reset_index()

f1['id'] = f1['id'].map(lambda x : int(x))

val2 = pd.merge(val,f1,how='left',on='id')

def weather(x) : 
    if (3 <= x) & (x <6) : 
        return('spring')
    elif (6 <= x) & (x <9) : 
        return('summer')
    elif (9 <= x) & (x <11) : 
        return('autumn')    
    else : return('winter')

val2['year'] = pd.to_datetime(val2['updt_date']).map(lambda x : x.year)
val2['month'] = pd.to_datetime(val2['updt_date']).map(lambda x : x.month)
val2['weather'] = val2['month'].apply(weather)

val2 = pd.get_dummies(data=val2,columns=['weather'])

val2['y'] = 0

train과 valid의 칼럼 개수 맞춰주기  
train에 있는 장르가 valid에 없을수도 있음!

In [ ]:
print(len(train2.columns), len(val2.columns))

232 232


In [ ]:
# train에는 있지만 val에는 없는 장르 추출!

lsss = []
for i in train2.columns:
  if i not in val2.columns:
    lsss.append(i) # 빈 리스트 안에 그 장르 넣기
lsss

[]

In [ ]:
# 해당 칼럼 0으로 만들어줌(칼럼 수 train, val 맞춰주려고 일부러 빈 칼럼 만드는듯!)
for j in lsss:
  val2[j] = 0

In [ ]:
val2 = val2[list(train2.columns)] # train2의 columns를 리스트로 넣어줌

In [ ]:
train2.columns

Index(['tags', 'id', 'plylst_title', 'songs', 'like_cnt', 'updt_date',
       'GN0101', 'GN0102', 'GN0103', 'GN0104',
       ...
       'GN2901', 'GN2902', 'GN2903', '장르없음', 'year', 'month', 'weather_autumn',
       'weather_spring', 'weather_summer', 'weather_winter'],
      dtype='object', length=232)

In [ ]:
print(len(train2.columns), len(val2.columns))

232 232


## Top Song
자주 등장한 노래 TOP 10000 뽑기!

In [ ]:
# songs가 들어간 빈 리스트 만듦
s = []
for song in train.songs.tolist():
    s += song # for문으로 리스트에 계속 삽입. 모든 노래가 s에 담김. 

#리스트 안의 원소의 개수를 세어줌
from collections import Counter 
r1 = dict(Counter(s)) # 각 노래가 몇번 등장했는지???? 딕셔너리 형태로 저장!
r1

{525514: 7,
 129701: 3,
 383374: 1,
 562083: 15,
 297861: 57,
 139541: 24,
 351214: 2,
 650298: 46,
 531057: 40,
 205238: 3,
 706183: 1,
 127099: 1,
 660493: 4,
 461973: 7,
 121455: 1,
 72552: 63,
 223955: 1,
 324992: 1,
 50104: 3,
 432406: 54,
 675945: 45,
 497066: 1111,
 120377: 15,
 389529: 311,
 244277: 38,
 461062: 135,
 696302: 34,
 442765: 18,
 532114: 853,
 586541: 417,
 33389: 204,
 244000: 102,
 692078: 18,
 37741: 29,
 645653: 37,
 571802: 48,
 200183: 30,
 61435: 20,
 204499: 30,
 41749: 12,
 129258: 11,
 413920: 8,
 117205: 3,
 6546: 1103,
 152422: 1207,
 602724: 105,
 425946: 197,
 173634: 77,
 631268: 2,
 409869: 1,
 97749: 2,
 395416: 390,
 103741: 4,
 181101: 8,
 472144: 7,
 414721: 23,
 75801: 26,
 315216: 24,
 192882: 43,
 383960: 244,
 548636: 84,
 83116: 188,
 276692: 84,
 166267: 363,
 186301: 17,
 354465: 39,
 256598: 23,
 233195: 9,
 666852: 101,
 686560: 20,
 556426: 14,
 142974: 238,
 331878: 72,
 195141: 52,
 32017: 41,
 617795: 30,
 396532: 22,
 623704: 26,


### 가장 많이 등장한 노래 순서대로 r_song 출력

In [ ]:
# .items() : 딕셔너리에서 key와 value 함께 뽑아내기
r_song = sorted(r1.items(), key=lambda x: -x[1]) # 큰 수부터 ?? 정렬 ???
# sorted : 딕셔너리는 순서가 없는데, sorted로 순서를 지정해주니까 튜플형태 리스트로 출력됨! 

r_song_top = r_song[:10000] # 몇곡 할지 정하기(top 10000개)
r_song

[(144663, 2175),
 (116573, 2121),
 (357367, 1981),
 (366786, 1919),
 (654757, 1647),
 (133143, 1633),
 (349492, 1599),
 (675115, 1598),
 (463173, 1544),
 (42155, 1540),
 (610933, 1538),
 (396828, 1538),
 (461341, 1526),
 (174749, 1512),
 (520093, 1508),
 (701557, 1507),
 (549178, 1501),
 (485155, 1492),
 (650494, 1479),
 (523521, 1476),
 (13281, 1473),
 (648628, 1470),
 (449244, 1468),
 (680366, 1467),
 (169984, 1463),
 (422915, 1455),
 (11657, 1452),
 (418935, 1446),
 (187047, 1437),
 (547967, 1434),
 (422077, 1432),
 (350309, 1425),
 (627363, 1424),
 (625875, 1421),
 (300087, 1415),
 (132994, 1414),
 (215411, 1408),
 (427724, 1407),
 (442014, 1403),
 (668128, 1402),
 (582252, 1380),
 (663256, 1378),
 (253755, 1375),
 (643628, 1375),
 (448116, 1368),
 (339802, 1364),
 (581799, 1363),
 (348200, 1363),
 (26083, 1355),
 (37748, 1354),
 (341513, 1338),
 (505036, 1334),
 (199262, 1333),
 (407828, 1325),
 (105140, 1324),
 (68348, 1324),
 (140867, 1323),
 (235773, 1320),
 (209993, 1313),
 (2

In [ ]:
list_song = list(dict(r_song_top).keys())
print(list_song)

[144663, 116573, 357367, 366786, 654757, 133143, 349492, 675115, 463173, 42155, 610933, 396828, 461341, 174749, 520093, 701557, 549178, 485155, 650494, 523521, 13281, 648628, 449244, 680366, 169984, 422915, 11657, 418935, 187047, 547967, 422077, 350309, 627363, 625875, 300087, 132994, 215411, 427724, 442014, 668128, 582252, 663256, 253755, 643628, 448116, 339802, 581799, 348200, 26083, 37748, 341513, 505036, 199262, 407828, 105140, 68348, 140867, 235773, 209993, 209135, 339124, 487911, 493762, 672550, 509998, 531820, 27469, 157055, 519391, 473514, 232874, 75842, 117595, 446812, 295250, 152422, 224921, 678762, 351342, 15318, 377243, 146989, 246531, 205179, 645489, 108004, 464051, 13198, 302646, 152475, 343974, 236393, 95323, 640657, 459256, 88503, 362966, 674160, 424813, 154858, 497066, 590012, 553171, 333595, 6546, 521739, 628232, 193232, 543820, 443914, 518420, 173943, 210647, 601037, 663905, 440160, 38261, 173798, 15124, 478754, 32385, 95485, 690604, 246984, 237407, 367963, 629738, 5

### for문으로 돌리기

In [ ]:
i = 0 # top1000 개중에 하나만 본 거임. 
# song에서 첫번째 곡을 가져와서 그 곡이 있는지 확인
# 이런 전처리를 validation set 도 똑같이 해줘야함. 

def fx(x) : 
    # 이미 리스트인데 딕셔너리로 바꿨다가 다시 리스트로 바꾼 이유 : key값만 가져올라고
    # (알고 싶은 값은 value가 아니라 노래 id인 key값이니까!)
    if list(dict(r_song_top).keys())[i] in x :  
        # 딕셔너리의 key, 자주 출현하는 리스트 안에 노래가 있으면 1 출력
        return(1)
    else : return 0

In [ ]:
train2['y'].value_counts()  # 타겟값은 0, 1
# 지금 첫번째 노래만 돌려서 그럼. 첫번째 노래가 각각 플레이리스트에 들어있는지 판단함

In [ ]:
train2 = pd.get_dummies(train2,columns=['year'])
val2 = pd.get_dummies(val2,columns=['year'])

### 2. LGBM 모델링

In [ ]:
train2.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,GN0101,GN0102,GN0103,GN0104,GN0105,GN0201,GN0202,GN0203,GN0204,GN0205,GN0301,GN0302,GN0303,GN0304,GN0401,GN0402,GN0403,GN0501,GN0502,GN0503,GN0504,GN0505,GN0506,GN0507,GN0508,GN0509,GN0601,GN0602,GN0603,GN0604,GN0605,GN0606,GN0701,GN0702,...,GN2506,GN2601,GN2602,GN2603,GN2701,GN2702,GN2703,GN2704,GN2801,GN2802,GN2803,GN2804,GN2805,GN2806,GN2901,GN2902,GN2903,장르없음,month,weather_autumn,weather_spring,weather_summer,weather_winter,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020
0,[락],61281,여행같은 음악,"[525514, 129701,...",71,2013-12-19 18:36...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945,...",1,2014-12-02 16:19...,0.104762,0.0,0.009524,0.028571,0.057143,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0000,0.000000,0.066667,0.019048,0.047619,0.0000,0.0,0.000000,0.0,0.019048,0.047619,0.076190,0.0,0.0,0.000000,0.047619,0.028571,0.000000,0.000000,...,0.000000,0.009524,0.009524,0.009524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있...","[83116, 276692, ...",17,2017-08-28 07:09...,0.025000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.037500,0.000000,0.03750,0.0,0.187500,0.0125,0.125000,0.150000,0.012500,0.050000,0.0125,0.0,0.000000,0.0,0.000000,0.150000,0.050000,0.0,0.0,0.000000,0.000000,0.050000,0.000000,0.000000,...,0.000000,0.012500,0.000000,0.012500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,"[연말, 눈오는날, 캐럴, 분...",147456,크리스마스 분위기에 흠뻑 취하...,"[394031, 195524,...",33,2019-12-05 15:15...,0.134615,0.0,0.009615,0.000000,0.057692,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0000,0.000000,0.076923,0.009615,0.038462,0.0000,0.0,0.028846,0.0,0.000000,0.076923,0.038462,0.0,0.0,0.000000,0.000000,0.038462,0.009615,0.009615,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610,...",9,2011-10-25 13:54...,0.034483,0.0,0.020690,0.000000,0.000000,0.289655,0.0,0.206897,0.055172,0.0,0.027586,0.006897,0.02069,0.0,0.006897,0.0000,0.006897,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.0,0.000000,0.000000,0.006897,0.0,0.0,0.006897,0.000000,0.000000,0.000000,0.000000,...,0.089655,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
